# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [23]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [24]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [25]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [26]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [27]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [28]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifykeyspace  
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)
    

#### Set Keyspace

In [29]:

try:
    session.set_keyspace('sparkifykeyspace')
except Exception as e:
    print(e)


### Query 1

#### Get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4 : 
##### Faithless 	| Music Matters (Mark Knight Dub) 	| 495.30731201171875

#### Create table :

In [30]:
#Description: 
#Here the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key; / 
#Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.

query1 = """SELECT artist, song, length FROM song_history_by_session WHERE sessionId = 338 AND itemInSession = 4"""
# create table named song_history_by_session for the 1st query

#1st we drop the table so wwe dont hav an issue
try:
    session.execute("DROP TABLE IF EXISTS song_history_by_session")
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS song_history_by_session"
query = query + "(sessionId BIGINT,itemInSession BIGINT,artist text,song text,length float, PRIMARY KEY (sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                   

#### Insert into table :

In [31]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history_by_session (sessionId ,itemInSession,artist ,song ,length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        #print(line[0])
        #we should cast the string to float 
        #for more reading : https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
        
        try:
            session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5]) ))
        except Exception as e:
            print(e)
    #close the file after we end with it
    f.close()

#### Display data :

In [33]:

try:
    res = session.execute(query1)
    for row in res:
        print (row.artist,"\t",row.song,"\t",row.length)
except Exception as e:
    print(e)
    



Faithless 	 Music Matters (Mark Knight Dub) 	 495.30731201171875


### Query 2

#### GET artist_name,song_name(sorted by itemInSession) and user for userid = 10, sessionid = 182 :
##### Down To The Bone |	 Keep On Keepin' On |	 0 |	 Sylvie |	 Cruz
##### Three Drives |	 Greece 2000 |	 1 |	 Sylvie |	 Cruz
##### Sebastien Tellier |	 Kilometer |	 2 |	 Sylvie |	 Cruz
##### Lonnie Gordon |	 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |	 3 |	 Sylvie |	 Cruz


 

#### Create table :

In [34]:
#Description : 
#Here  Primary Key is a combinision from 3 fields : partition key is combinition of 2 fileds to store all seassons with the same user / 
#in the same node and we have the filed itemInSession as clustering key so we have the rows resiult are sorted by itemInSession

#
query2 = """SELECT artist_name, song_name, itemInSession, user_first_name, user_last_name FROM song_history_by_userID WHERE userid = 10 AND sessionid = 182"""
# create table named  for the song_history_by_userID

#1st we drop the table so wwe dont hav an issue
try:
    session.execute("DROP TABLE IF EXISTS song_history_by_userID")
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS song_history_by_userID"
query = query + "(userid int,sessionid BIGINT,itemInSession BIGINT,artist_name text, song_name text,  user_first_name text,user_last_name text, PRIMARY KEY ((userid,sessionid),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                   

#### Insert into table :

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history_by_userID (userid,sessionid,itemInSession,artist_name, song_name, user_first_name,user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        #print(line[0])
        #we should cast the string to float 
        #for more reading : https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
        
        try:
            #print((line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))
            session.execute(query, (int(line[10]), int(line[8]),int(line[3]),line[0], line[9],  line[1], line[4]))
        except Exception as e:
            print(e)
    #close the file after we end with it
    f.close()
        

#### Display data :

In [36]:

try:
    res = session.execute(query2)
    for row in res:
        print (row.artist_name,"\t",row.song_name,"\t",row.iteminsession,"\t",row.user_first_name,"\t",row.user_last_name)
except Exception as e:
    print(e)
    


Down To The Bone 	 Keep On Keepin' On 	 0 	 Sylvie 	 Cruz
Three Drives 	 Greece 2000 	 1 	 Sylvie 	 Cruz
Sebastien Tellier 	 Kilometer 	 2 	 Sylvie 	 Cruz
Lonnie Gordon 	 Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 	 3 	 Sylvie 	 Cruz


### Query 3

#### Get all users who listened to the song 'All Hands Against His Own' :
##### Jacqueline |	 Lynch
##### Tegan |	 Levine
##### Sara |	 Johnson


 

#### Create table :

In [37]:
#Description : 
#Here the Primary key Compose with 2 fields : song_name is the partition key and we need to add the user_id to unqiue identify rows /
#in this table

query3 = """SELECT user_first_name, user_last_name FROM song_history_by_songName WHERE song_name = 'All Hands Against His Own'"""
# create table named song_history_by_songName for the 3st query

#1st we drop the table so wwe dont hav an issue
try:
    session.execute("DROP TABLE IF EXISTS song_history_by_songName")
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS song_history_by_songName"
query = query + "(song_name text,userid int,user_first_name text, user_last_name text,  PRIMARY KEY (song_name,userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                   

#### Insert into table :

In [38]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history_by_songName (song_name,userid,user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        #print(line[0])
        #we should cast the string to float 
        #for more reading : https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
        
        try:
            session.execute(query, (line[9],int(line[10]),line[1], line[4]))
        except Exception as e:
            print(e)
    #close the file after we end with it
    f.close()

#### Display data :

In [39]:

try:
    res = session.execute(query3)
    for row in res:
        print (row.user_first_name,"\t",row.user_last_name)
except Exception as e:
    print(e)
    



Jacqueline 	 Lynch
Tegan 	 Levine
Sara 	 Johnson


### delete tables :

In [40]:
try:
    session.execute("DROP TABLE IF EXISTS song_history_by_session")
except Exception as e:
    print(e)

In [41]:
try:
    session.execute("DROP TABLE IF EXISTS song_history_by_userID")
except Exception as e:
    print(e)

In [42]:
try:
    session.execute("DROP TABLE IF EXISTS song_history_by_songName")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [43]:
session.shutdown()
cluster.shutdown()

### resources used: 


In [44]:
##https://knowledge.udacity.com/questions/46515
##https://www.tutorialspoint.com/cassandra/cassandra_create_keyspace.htm
##https://stackoverflow.com/questions/55290883/invalid-string-constant-error-in-apache-cassandra-using-python
##https://stackoverflow.com/questions/30418185/order-by-any-field-in-cassandra
##https://www.tutorialspoint.com/python/file_close.htm
##https://docs.python.org/3/library/glob.html
##Udacity review 